In [174]:
import re
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# machine learning using all the data

In [294]:
idx2word = dict()

with open('C:/Users/Administrator/Desktop/MBS/Module3/Text and Web Analytics/assignment/our_dataset/aclImdb/imdb.vocab', 'r') as f:
    for idx, line in enumerate(f):
        # remove '\n' at the end of each line
        word = re.sub(r"\n", '', line)
        idx2word[idx] = word
        
word_rating = dict()

# change lexicon for the line below if needed
with open('C:/Users/Administrator/Desktop/MBS/Module3/Text and Web Analytics/assignment/My work/sort_reviews_by_genres/genres/train/lexicon/Horror.txt', 'r') as f:
#with open('C:/Users/Administrator/Desktop/MBS/Module3/Text and Web Analytics/assignment/our_dataset/aclImdb/imdbEr.txt', 'r') as f:    
    for idx, line in enumerate(f):
        # remove '\n' at the end of each line
        rating = re.sub(r"\n", '', line)
        word_rating[idx2word[idx]] = float(rating)

# word_rating = {word:rating, word:rating, .........}

# BOW models (General model 1)

In [166]:
# training data in BOW
f = open('C:/Users/Administrator/Desktop/MBS/Module3/Text and Web Analytics/assignment/our_dataset/aclImdb/train/labeledBow.feat','r')
X_train_bow = []
y_train_bow = []

for line in f:
    fields=re.split(r'\s+',line)
    # Positive reviews have 7 or more stars, negative reviews have <4
    if (int(fields[0])>=7):
        y_train_bow.append("1")
    else:
        y_train_bow.append("0")

    # this_BOW will store all of the term frequencies for this line in a dictionary
    # let's start by attempting to process all of the TDM entries
    this_BOW = {}
    for i in range(1, len(fields)-1):
        index, val = re.split(r':',fields[i])
        word = idx2word[int(index)]
        this_BOW[word] = int(val)
        
    X_train_bow.append(this_BOW)

f.close()

In [150]:
# testing
f = open('C:/Users/Administrator/Desktop/MBS/Module3/Text and Web Analytics/assignment/our_dataset/aclImdb/test/labeledBow.feat','r')
X_test_bow = []
y_test_bow = []

for line in f:
    fields=re.split(r'\s+',line)
    # Positive reviews have 7 or more stars, negative reviews have <4
    if (int(fields[0])>=7):
            y_test_bow.append("1")
    else:
            y_test_bow.append("0")

    # this_BOW will store all of the term frequencies for this line in a dictionary
    # let's start by attempting to process all of the TDM entries
    this_BOW = {}
    for i in range(1, len(fields)-1):
        index, val = re.split(r':',fields[i])
        word = idx2word[int(index)]
        this_BOW[word] = int(val)
        
    X_test_bow.append(this_BOW)

f.close()

In [151]:
vectorizer = DictVectorizer()
# Only use fit_transform() for the training set to ensure that feature set is determined by the training set
X_train_bow = vectorizer.fit_transform(X_train_bow)
X_test_bow = vectorizer.transform(X_test_bow)

In [152]:
# note that the third and forth models are both failed to converge
clfs = [KNeighborsClassifier(),RandomForestClassifier(),MultinomialNB(),LinearSVC(),LogisticRegression()]

for clf in clfs:
    clf.fit(X_train_bow, y_train_bow)
    # infer the label of each instance in the test set
    predictions = clf.predict(X_test_bow)
    print('Accuracy: ', accuracy_score(y_test_bow, predictions))
    print(classification_report(y_test_bow, predictions))

Accuracy:  0.6254
              precision    recall  f1-score   support

           0       0.65      0.54      0.59     12500
           1       0.61      0.71      0.66     12500

    accuracy                           0.63     25000
   macro avg       0.63      0.63      0.62     25000
weighted avg       0.63      0.63      0.62     25000

Accuracy:  0.84172
              precision    recall  f1-score   support

           0       0.84      0.84      0.84     12500
           1       0.84      0.84      0.84     12500

    accuracy                           0.84     25000
   macro avg       0.84      0.84      0.84     25000
weighted avg       0.84      0.84      0.84     25000

Accuracy:  0.8136
              precision    recall  f1-score   support

           0       0.78      0.88      0.82     12500
           1       0.86      0.75      0.80     12500

    accuracy                           0.81     25000
   macro avg       0.82      0.81      0.81     25000
weighted avg       

C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Accuracy:  0.84852
              precision    recall  f1-score   support

           0       0.84      0.86      0.85     12500
           1       0.86      0.84      0.85     12500

    accuracy                           0.85     25000
   macro avg       0.85      0.85      0.85     25000
weighted avg       0.85      0.85      0.85     25000

Accuracy:  0.86972
              precision    recall  f1-score   support

           0       0.87      0.87      0.87     12500
           1       0.87      0.87      0.87     12500

    accuracy                           0.87     25000
   macro avg       0.87      0.87      0.87     25000
weighted avg       0.87      0.87      0.87     25000



C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Sum of word scores model (General model 2 and genre specific models)

In [295]:
# training using sum of rating for each review

#f = open('C:/Users/Administrator/Desktop/MBS/Module3/Text and Web Analytics/assignment/our_dataset/aclImdb/train/labeledBow.feat','r')

# above for general model and below for genre specific models

f = open('C:/Users/Administrator/Desktop/MBS/Module3/Text and Web Analytics/assignment/My work/sort_reviews_by_genres/genres/train/bow/Horror.txt','r')
X_train = []
y_train = []

for line in f:
    fields=re.split(r'\s+',line)
    # Positive reviews have 7 or more stars, negative reviews have <4
    if (int(fields[0])>=7):
        y_train.append("1")
    else:
        y_train.append("0")

    # this_BOW will store all of the term frequencies for this line in a dictionary
    # let's start by attempting to process all of the TDM entries
    sum_rating = 0
    for i in range(1, len(fields)-1):
        index, val = re.split(r':',fields[i])
        word = idx2word[int(index)]
        rating = word_rating[word] * int(val)
        sum_rating += rating
        
    X_train.append(sum_rating)

f.close()

In [296]:
# extra features 1&2
# if run general model, this cell should be skipped

count_path = r'C:/Users/Administrator/Desktop/MBS/Module3/Text and Web Analytics/assignment/My work/sort_reviews_by_genres/genres/train/word_count/'
count1_path = r'C:/Users/Administrator/Desktop/MBS/Module3/Text and Web Analytics/assignment/My work/sort_reviews_by_genres/genres/train/exclamation_count/'
#genre_list = ['Drama', 'Comedy', 'Action', 'Horror', 'Romance', 'Crime', 'Thriller', 'Adventure', 'Fantasy', 'Mystery', 'Sci-Fi', \
#              'Family', 'Biography', 'Music', 'Animation', 'Documentary', 'War', 'History', 'Short', 'Sport', 'Musical', 'Western', \
#              'Film-Noir', 'Reality-TV', 'Talk-Show', 'Adult', 'Game-Show', 'News']

with open(count_path + 'Horror.txt', 'r') as f:
    temp = []
    for count in f:
        count = count.strip()
        temp.append(int(count))

with open(count1_path + 'Horror.txt', 'r') as f:
    temp1 = []
    for count1 in f:
        count1 = count1.strip()
        temp1.append(int(count1))

X_train = pd.DataFrame({'sum_score':X_train, 'word_count':temp, 'exclamation_count':temp1})

print(X_train)


      sum_score  word_count  exclamation_count
0      6.455086         171                  0
1     11.200551         108                  1
2     22.651711         340                  0
3     53.365204         436                  0
4     22.406493         324                  0
...         ...         ...                ...
4326  -8.390992         476                  8
4327 -21.962934         275                  0
4328 -56.947705         779                  1
4329  -8.604051         562                  3
4330 -16.581549         232                  0

[4331 rows x 3 columns]


In [297]:
# testing

#f = open('C:/Users/Administrator/Desktop/MBS/Module3/Text and Web Analytics/assignment/our_dataset/aclImdb/test/labeledBow.feat','r')

# above for general model and below for genre specific models

f = open('C:/Users/Administrator/Desktop/MBS/Module3/Text and Web Analytics/assignment/My work/sort_reviews_by_genres/genres/test/bow/Horror.txt','r')
X_test = []
y_test = []

for line in f:
    fields=re.split(r'\s+',line)
    # Positive reviews have 7 or more stars, negative reviews have <4
    if (int(fields[0])>=7):
            y_test.append("1")
    else:
            y_test.append("0")

    # this_BOW will store all of the term frequencies for this line in a dictionary
    # let's start by attempting to process all of the TDM entries
    sum_rating = 0
    for i in range(1, len(fields)-1):
        index, val = re.split(r':',fields[i])
        word = idx2word[int(index)]
        rating = word_rating[word] * int(val)
        sum_rating += rating
        
    X_test.append(sum_rating)

f.close()

In [298]:
# extra features 1&2
# if run general model, this cell should be skipped

count_path = r'C:/Users/Administrator/Desktop/MBS/Module3/Text and Web Analytics/assignment/My work/sort_reviews_by_genres/genres/test/word_count/'
count1_path = r'C:/Users/Administrator/Desktop/MBS/Module3/Text and Web Analytics/assignment/My work/sort_reviews_by_genres/genres/test/exclamation_count/'
#genre_list = ['Drama', 'Comedy', 'Action', 'Horror', 'Romance', 'Crime', 'Thriller', 'Adventure', 'Fantasy', 'Mystery', 'Sci-Fi', \
#              'Family', 'Biography', 'Music', 'Animation', 'Documentary', 'War', 'History', 'Short', 'Sport', 'Musical', 'Western', \
#              'Film-Noir', 'Reality-TV', 'Talk-Show', 'Adult', 'Game-Show', 'News']

with open(count_path + 'Horror.txt', 'r') as f:
    temp = []
    for count in f:
        count = count.strip()
        temp.append(int(count))

with open(count1_path + 'Horror.txt', 'r') as f:
    temp1 = []
    for count1 in f:
        count1 = count1.strip()
        temp1.append(int(count1))

X_test = pd.DataFrame({'sum_score':X_test, 'word_count':temp, 'exclamation_count':temp1})

print(X_test)
        


      sum_score  word_count  exclamation_count
0     -1.020373         183                  0
1     49.220538         674                  1
2     12.794878         189                  0
3      6.889241         134                  0
4      1.052985         301                  0
...         ...         ...                ...
4492   2.850117         317                  0
4493  -0.283095         205                  0
4494   6.139030         235                  1
4495  -3.269744         209                  0
4496  -4.291644         326                  1

[4497 rows x 3 columns]


In [299]:
#X_train = np.array(X_train).reshape(-1, 1)
#X_test = np.array(X_test).reshape(-1, 1)

# above for general model and below for genre specific models

X_train = np.array(X_train)
X_test = np.array(X_test)

In [300]:
# note that the third and forth models are both failed to converge

clf = LogisticRegression()

clf.fit(X_train, y_train)
# infer the label of each instance in the test set
predictions = clf.predict(X_test)
print('Accuracy: ', accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))

importance = clf.coef_[0]
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))

Accuracy:  0.8003113186568823
              precision    recall  f1-score   support

           0       0.82      0.96      0.88      3500
           1       0.63      0.24      0.34       997

    accuracy                           0.80      4497
   macro avg       0.72      0.60      0.61      4497
weighted avg       0.77      0.80      0.76      4497

Feature: 0, Score: 0.08467
Feature: 1, Score: -0.00044
Feature: 2, Score: -0.01519
